<a href="https://colab.research.google.com/github/LI3ARA/FYIRP/blob/main/Notebooks/25/VQA_RAG/Googl_colab/host_llm_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working code

Colab:

- Run llava-cli in a Flask or FastAPI app

- Expose it using ngrok so VS Code can reach it

Steps:
- Load the LLaVA model using llava.cpp

- Start a Flask server in Colab

- Expose the server via ngrok

- Accept an image + prompt, run llava-cli, and return the output

### Install necessary files and libraries

In [8]:
# Install dependencies
!pip install flask llama-cpp-python --quiet

# Install and set up ngrok
!wget -q -O ngrok.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok.tgz
!chmod +x ngrok


In [9]:
# Add your ngrok auth token (looks like 2N...)
!./ngrok config add-authtoken 2vLvftpzesWBGHU32SAnWOWmryA_CvhQGZ7cyfpk6TEkPG1b

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
%cd /content
!rm -rf llava.cpp
!sudo apt-get update
!sudo apt-get install -y build-essential cmake git

/content
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Don

### Clone llama.cpp from ggml-org

In [51]:
# Clone & build LLaVA
!rm -rf llava.cpp
!git clone --recursive https://github.com/ggml-org/llama.cpp.git
%cd llama.cpp

# CMake build with Vision Support
!cmake -S . -B build -DGGML_CUDA=on -DLLAMA_LLAVA_VISION=on
!cmake --build build -j

CMake Warning at CMakeLists.txt:107 (message):
  LLAMA_CUDA is deprecated and will be removed in the future.

  Use GGML_CUDA instead

Call Stack (most recent call first):
  CMakeLists.txt:113 (llama_option_depr)


-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- x86 detected
-- Adding CPU backend variant ggml-cpu: -march=native 
-- CUDA Toolkit found
-- Using CUDA architectures: native
-- CUDA host compiler is GNU 11.4.0

-- Including CUDA backend
-- Configuring done (0.5s)
-- Generating done (0.3s)
CMake Warning:
  Manually-specified variables were not used by the project:

    LLAMA_LLAVA_VISION


-- Build files have been written to: /content/llama.cpp/llama.cpp/llama.cpp/build
[  0%] Built target build_info
[  0%] Built target sha256
[  1%] Built target sha1
[  2%] Built target xxhash
[  5%] Built target ggml-base
[ 10%] Built target ggml-cpu
[ 11%]

In [52]:
#  Verify Build Output
!ls build/bin

libggml-base.so		       llama-q8dot
libggml-cpu.so		       llama-quantize
libggml-cuda.so		       llama-quantize-stats
libggml.so		       llama-qwen2vl-cli
libllama.so		       llama-retrieval
libllava_shared.so	       llama-run
libmtmd_shared.so	       llama-save-load-state
llama-batched		       llama-server
llama-batched-bench	       llama-simple
llama-bench		       llama-simple-chat
llama-cli		       llama-speculative
llama-convert-llama2c-to-ggml  llama-speculative-simple
llama-cvector-generator        llama-tokenize
llama-embedding		       llama-tts
llama-eval-callback	       llama-vdot
llama-export-lora	       test-arg-parser
llama-gbnf-validator	       test-autorelease
llama-gemma3-cli	       test-backend-ops
llama-gen-docs		       test-barrier
llama-gguf		       test-c
llama-gguf-hash		       test-chat
llama-gguf-split	       test-chat-template
llama-gritlm		       test-gguf
llama-imatrix		       test-grammar-integration
llama-infill		       test-grammar-parser
llama-llava-cli		

In [53]:
!./build/bin/llama-llava-cli

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
build: 5142 (80f19b41) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu

 example usage:

     ./build/bin/llama-llava-cli -m <llava-v1.5-7b/ggml-model-q5_k.gguf> --mmproj <llava-v1.5-7b/mmproj-model-f16.gguf> --image <path/to/an/image.jpg> --image <path/to/another/image.jpg> [--temp 0.1] [-p "describe the image in detail."]

 note: a lower temperature value like 0.1 is recommended for better quality.


In [54]:
!find . -name "llama-llava-cli"

./build/bin/llama-llava-cli


### Download LLaVA GGUF Model and Image

In [19]:
# Download GGUF Model from Second State (7B, Q4_K_M)
# !mkdir -p models
# !wget -O models/llava-v1.5-7b-Q4_K_M.gguf https://huggingface.co/second-state/Llava-v1.5-7B-GGUF/resolve/main/llava-v1.5-7b-Q4_K_M.gguf

--2025-04-16 05:30:07--  https://huggingface.co/second-state/Llava-v1.5-7B-GGUF/resolve/main/llava-v1.5-7b-Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.121, 13.35.202.40, 13.35.202.97, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.121|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/43/d0/43d0efd222e373f385d959b6e01e13661ead062fc9fff5a8adb0f53e8dba9f6c/2687b20ac8b7a23f6c70296d5b1e7f908fef2ce4769ecdebd1bb9503528a75bf?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llava-v1.5-7b-Q4_K_M.gguf%3B+filename%3D%22llava-v1.5-7b-Q4_K_M.gguf%22%3B&Expires=1744785007&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDc4NTAwN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQzL2QwLzQzZDBlZmQyMjJlMzczZjM4NWQ5NTliNmUwMWUxMzY2MWVhZDA2MmZjOWZmZjVhOGFkYjBmNTNlOGRiYTlmNmMvMjY4N2IyMGFjOGI3YTIzZjZjNzAyOTZkNWIxZTdmOTA4ZmVmMmNlNDc2OWVjZGViZD

In [55]:
# !mkdir -p models
!mkdir -p models
!wget -O models/llava.gguf https://huggingface.co/second-state/Llava-v1.5-7B-GGUF/resolve/main/llava-v1.5-7b-Q5_K_M.gguf


--2025-04-16 06:35:02--  https://huggingface.co/second-state/Llava-v1.5-7B-GGUF/resolve/main/llava-v1.5-7b-Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.121, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/43/d0/43d0efd222e373f385d959b6e01e13661ead062fc9fff5a8adb0f53e8dba9f6c/217d0d6a0069af51040ea4bbd1761d038e378e9dfd1851cb5be57f405d01920d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llava-v1.5-7b-Q5_K_M.gguf%3B+filename%3D%22llava-v1.5-7b-Q5_K_M.gguf%22%3B&Expires=1744788902&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDc4ODkwMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQzL2QwLzQzZDBlZmQyMjJlMzczZjM4NWQ5NTliNmUwMWUxMzY2MWVhZDA2MmZjOWZmZjVhOGFkYjBmNTNlOGRiYTlmNmMvMjE3ZDBkNmEwMDY5YWY1MTA0MGVhNGJiZDE3NjFkMDM4ZTM3OGU5ZGZkMTg1MWNiNWJ

In [40]:
# Confirm the path of the model
!ls -lh models/llava.gguf

-rw-r--r-- 1 root root 4.5G Feb 28  2024 models/llava.gguf


### Download the projection Layer

In [44]:
# Vision projection layer
!wget -O models/mmproj.gguf https://huggingface.co/second-state/Llava-v1.5-7B-GGUF/resolve/main/llava-v1.5-7b-mmproj-model-f16.gguf

--2025-04-16 06:20:55--  https://huggingface.co/second-state/Llava-v1.5-7B-GGUF/resolve/main/llava-v1.5-7b-mmproj-model-f16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.40, 13.35.202.121, 13.35.202.34, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/43/d0/43d0efd222e373f385d959b6e01e13661ead062fc9fff5a8adb0f53e8dba9f6c/50da4e5b0a011615f77686f9b02613571e65d23083c225e107c08c3b1775d9b1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llava-v1.5-7b-mmproj-model-f16.gguf%3B+filename%3D%22llava-v1.5-7b-mmproj-model-f16.gguf%22%3B&Expires=1744788055&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDc4ODA1NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQzL2QwLzQzZDBlZmQyMjJlMzczZjM4NWQ5NTliNmUwMWUxMzY2MWVhZDA2MmZjOWZmZjVhOGFkYjBmNTNlOGRiYTlmNmMvNTBkYTRlNWIwYTAxMTYxNWY3NzY4NmY5YjAyN

## Test The Model - Vision

In [46]:
!wget -O cat.jpg https://upload.wikimedia.org/wikipedia/commons/3/3a/Cat03.jpg

--2025-04-16 06:24:47--  https://upload.wikimedia.org/wikipedia/commons/3/3a/Cat03.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279603 (273K) [image/jpeg]
Saving to: ‘cat.jpg’

cat.jpg             100%[===================>] 273.05K  --.-KB/s    in 0.01s   

2025-04-16 06:24:47 (27.6 MB/s) - ‘cat.jpg’ saved [279603/279603]



In [57]:
!build/bin/llama-llava-cli \
  -m models/llava.gguf \
  --mmproj models/mmproj.gguf \
  --image cat.jpg \
  -p "Describe this image in detail." \
  --temp 0.1

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
build: 5142 (80f19b41) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
llama_model_load_from_file_impl: using device CUDA0 (NVIDIA A100-SXM4-40GB) - 40082 MiB free
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/llava.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_m

### Download the LLM - Mistral

In [2]:
!./build/bin/llama-cli -hf lmstudio-community/Mistral-7B-Instruct-v0.3-GGUF -p "What's the capital of France?"

/bin/bash: line 1: ./build/bin/llama-cli: No such file or directory


In [59]:
!mkdir -p models
# !./build/bin/llama-cli -hf lmstudio-community/Mistral-7B-Instruct-v0.3-GGUF


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
common_download_file_single: no previous model file found /root/.cache/llama.cpp/lmstudio-community_Mistral-7B-Instruct-v0.3-GGUF_Mistral-7B-Instruct-v0.3-Q4_K_M.gguf
curl_perform_with_retry: Trying to download from https://huggingface.co/lmstudio-community/Mistral-7B-Instruct-v0.3-GGUF/resolve/main/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf (attempt 1 of 3)...
common_download_file_single: trying to download model from https://huggingface.co/lmstudio-community/Mistral-7B-Instruct-v0.3-GGUF/resolve/main/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf to /root/.cache/llama.cpp/lmstudio-community_Mistral-7B-Instruct-v0.3-GGUF_Mistral-7B-Instruct-v0.3-Q4_K_M.gguf (server_etag:"f13896f73ac927396749ee484a4a1691-1", server_last_modified:Wed, 22 May 2024 19:28:46 GMT)...
curl_perform_with_retry: Trying to download from ht



---



## Flask API with Dual Endpoints

In [3]:
from flask import Flask, request, jsonify
import subprocess
import uuid
import os

app = Flask(__name__)

@app.route('/')
def home():
    return "🧠 API is running! Use /chat (Mistral) or /vision (LLaVA)"

@app.route('/chat', methods=['POST'])
def chat():
    prompt = request.json.get("prompt", "")
    if not prompt:
        return jsonify({"error": "Prompt is required"}), 400

    result = subprocess.run([
        "./build/bin/llama-cli",
        "-m", "models/mistral.gguf",
        "-p", prompt,
        "--temp", "0.7"
    ], capture_output=True, text=True)

    output = result.stdout.strip().split("\n")[-1]
    return jsonify({"response": output})

@app.route('/vision', methods=['POST'])
def vision():
    img = request.files["image"]
    prompt = request.form.get("prompt", "Describe the image")
    img_path = f"/tmp/{uuid.uuid4().hex}.jpg"
    img.save(img_path)

    result = subprocess.run([
        "./build/bin/llama-llava-cli",
        "-m", "models/llava.gguf",
        "--mmproj", "models/mmproj.gguf",
        "--image", img_path,
        "-p", prompt,
        "--temp", "0.1"
    ], capture_output=True, text=True)

    os.remove(img_path)
    output = result.stdout.strip().split("\n")[-1]
    return jsonify({"response": output})


In [6]:
#  Run the Flask App in Colab
import threading # to in the background and use other cells

def run_app():
    app.run(host="0.0.0.0", port=5000)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [10]:
import subprocess
import time
import requests

# Start ngrok in background
ngrok = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(5)

# Fetch the public URL
try:
    r = requests.get("http://localhost:4040/api/tunnels")
    public_url = r.json()["tunnels"][0]["public_url"]
    print(" Public API:", public_url)
except Exception as e:
    print("Failed to get ngrok URL:", e)



 Public API: https://7b9e-34-142-149-123.ngrok-free.app




---



# From VS Code

## Use /chat endpoint

In [ ]:
response = requests.post(
    "https://your-ngrok-url.ngrok.io/chat",
    json={"prompt": "What is retrieval-augmented generation?"}
)
print(response.json()["response"])


## Use /vision endpoint

In [12]:
files = {"image": open("cat.jpg", "rb")}
data = {"prompt": "What is this image about?"}

response = requests.post("https://your-ngrok-url.ngrok.io/vision", files=files, data=data)
print(response.json()["response"])


FileNotFoundError: [Errno 2] No such file or directory: 'cat.jpg'



---



### Start Flask API

In [ ]:
# Start Flask API
import os
import subprocess
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return '👋 LLaVA API is running! Use POST /generate'

@app.route('/generate', methods=['POST'])
def generate():
    img_file = request.files['image']
    prompt = request.form.get('prompt', '')

    # Save the uploaded image
    img_path = "input.jpg"
    img_file.save(img_path)

    # Run LLaVA CLI on the image and prompt
    result = subprocess.run([
        "./llava-cli",
        "-m", "models/llava.gguf",
        "--image", img_path,
        "-p", prompt
    ], capture_output=True, text=True)

    output = result.stdout.strip().split("\n")[-1]
    return jsonify({"response": output})

app.run(port=5000)


In [ ]:
import subprocess, time, requests
from llama_cpp import Llama
from flask import Flask, request, jsonify

In [ ]:
# Start ngrok on port 5000 and fetch public URL

ngrok_proc = subprocess.Popen(['./ngrok', 'http', '5000'])
time.sleep(5)
try:
    res = requests.get('http://localhost:4040/api/tunnels')
    url = res.json()['tunnels'][0]['public_url']
    print('✅ Public URL:', url)
except Exception as e:
    print('❌ Could not get ngrok URL:', e)

✅ Public URL: https://0232-34-16-171-130.ngrok-free.app


In [ ]:
# Load model
llm = Llama.from_pretrained(
    repo_id='TheBloke/Llama-2-13B-chat-GGUF',
    filename='llama-2-13b-chat.Q3_K_S.gguf',
    n_gpu_layers=40,
    n_ctx=2048
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q3_K_S.gguf:   0%|          | 0.00/5.66G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/./llama-2-13b-chat.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimensio

In [ ]:
# Start Flask app with LLaMA endpoint

app = Flask(__name__)

@app.route('/')
def home():
    return '✅ LLaMA Flask API is running! Use /generate'

@app.route('/generate', methods=['POST'])
def generate():
    prompt = request.json.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'Prompt is required'}), 400
    output = llm(prompt, max_tokens=200)
    return jsonify({'response': output['choices'][0]['text'].strip()})

app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
llama_perf_context_print:        load time =     602.34 ms
llama_perf_context_print: prompt eval time =     602.16 ms /     4 tokens (  150.54 ms per token,     6.64 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     603.06 ms /     5 tokens
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:11:07] "POST /generate HTTP/1.1" 200 -
Llama.generate: 3 prefix-match hit, remaining 1 prompt tokens to eval
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:12:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:12:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:12:19] "GET /generate HTTP/1.1" 405 -
llama_perf_contex



---



# Step by step working- test code

Without llm , just using a test text to pass

## In colab

### Install Dependencies

In [ ]:
!pip install ngrok


In [ ]:
!wget -q -O ngrok.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok.tgz
!chmod +x ngrok

# 🧠 Your ngrok auth token goes here (not your API key)
!./ngrok config add-authtoken <here>


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
! pip install llama_cpp

ERROR: Could not find a version that satisfies the requirement llama_cpp (from versions: none)
ERROR: No matching distribution found for llama_cpp


---

###

In [ ]:
import subprocess, time, requests
from flask import Flask

# Start ngrok on port 5000
ngrok_proc = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(5)

# Try to get the public tunnel URL
try:
    tunnels = requests.get("http://localhost:4040/api/tunnels").json()
    public_url = tunnels["tunnels"][0]["public_url"]
    print("✅ ngrok public URL:", public_url)
except Exception as e:
    print("❌ ngrok failed:", e)


✅ ngrok public URL: https://d67b-104-199-189-124.ngrok-free.app


In [ ]:
app = Flask(__name__)

@app.route("/")
def index():
    return "✅ LLaMA Flask API is running!"

app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
# Check if ngrok is alive
import subprocess, time, requests

# Start ngrok (it runs in the background)
ngrok_proc = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(4)

# Try to read the tunnel info
try:
    res = requests.get("http://localhost:4040/api/tunnels")
    print("🔍 Raw response:", res.text)
    data = res.json()
    print("✅ Public URL:", data["tunnels"][0]["public_url"])
except Exception as e:
    print("❌ Could not fetch ngrok tunnel:", e)


🔍 Raw response: {"tunnels":[{"name":"command_line","ID":"e2a1c3a7137a897877e63bb2f6a64125","uri":"/api/tunnels/command_line","public_url":"https://f4cf-104-199-189-124.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}

✅ Public URL: https://f4cf-104-199-189-124.ngrok-free.app


In [ ]:
requests.get("http://localhost:4040/api/tunnels").json()

{'tunnels': [{'name': 'command_line',
   'ID': 'e2a1c3a7137a897877e63bb2f6a64125',
   'uri': '/api/tunnels/command_line',
   'public_url': 'https://f4cf-104-199-189-124.ngrok-free.app',
   'proto': 'https',
   'config': {'addr': 'http://localhost:5000', 'inspect': True},
   'metrics': {'conns': {'count': 0,
     'gauge': 0,
     'rate1': 0,
     'rate5': 0,
     'rate15': 0,
     'p50': 0,
     'p90': 0,
     'p95': 0,
     'p99': 0},
    'http': {'count': 0,
     'rate1': 0,
     'rate5': 0,
     'rate15': 0,
     'p50': 0,
     'p90': 0,
     'p95': 0,
     'p99': 0}}}],
 'uri': '/api/tunnels'}

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/")
def home():
    return "✅ Hello from Flask with ngrok!"

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json()
    prompt = data.get("prompt", "No prompt provided.")
    return jsonify({
        "response": f"🧪 Test response to: '{prompt}'"
    })

app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:23:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:23:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:24:24] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:29:44] "GET / HTTP/1.1" 200 -


## In VS code

In [ ]:
# In vs code
import requests

url = "https://f4cf-104-199-189-124.ngrok-free.app/generate"  # Replace with your real one
response = requests.post(url, json={"prompt": "Hello world!"})
print("🧠 Response:", response.json()["response"])



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

---

In [ ]:
! pip install pyngrok

In [ ]:
# Use ngrok to expose your Flask server
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


t=2025-04-16T10:17:29+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-04-16T10:17:29+0530 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-04-16T10:17:29+0530 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.